In [ ]:
# Imports
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

sns.set_style('darkgrid')
print('TensorFlow version:', tf.__version__)

## 1) Parámetros y rutas
Ajusta `BASE_DIR` si ejecutas en otro entorno. En Kaggle los datos estarán en el directorio relativo creado por `DataSimplification.ipynb` (arriba de este notebook).

In [ ]:
BASE_DIR = '..'  # nivel por encima del notebook actual
TRAIN_DIR = os.path.join(BASE_DIR, 'Cancer_train')
VAL_DIR = os.path.join(BASE_DIR, 'Cancer_validation')
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 200

print('Train dir:', TRAIN_DIR)
print('Val dir:', VAL_DIR)

## 2) ImageDataGenerator (augmentación para training, solo rescale para validación)

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=20,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1./255,
    shear_range=0.2,
    fill_mode='nearest',
    zoom_range=0.2
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(TRAIN_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary')
val_generator = val_datagen.flow_from_directory(VAL_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='binary')

## 3) Definición del modelo (CNN)
Arquitectura tomada del notebook original (equivalente al bloque comentado).

In [ ]:
def build_cnn(input_shape=(128,128,3)):
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))

    model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

cancer_model = build_cnn()
cancer_model.summary()

## 4) Compilación y callbacks
Usamos `binary_crossentropy`, `Adam` y `EarlyStopping` + `ModelCheckpoint` para guardar el mejor modelo en `best_model.h5`.

In [ ]:
cancer_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss')

print('Callbacks preparados: EarlyStopping y ModelCheckpoint')

## 5) Entrenamiento
Entrenamos la CNN y guardamos el mejor modelo automáticamente en `best_model.h5`.

In [ ]:
history = cancer_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[early_stop, checkpoint],
    verbose=1
)

# Después del entrenamiento, el mejor modelo está en best_model.h5 (ModelCheckpoint)
# Si quieres, guardamos también con un nombre final estándar:
cancer_model.save('modelo_breast_cancer_final.h5')
print('Modelo final guardado como modelo_breast_cancer_final.h5')

## 6) Descargar el `.h5` (Kaggle)
En Kaggle puedes usar `FileLink` para facilitar la descarga del archivo generado.

In [ ]:
try:
    from IPython.display import FileLink, display
    if os.path.exists('modelo_breast_cancer_final.h5'):
        display(FileLink('modelo_breast_cancer_final.h5'))
    else:
        print('No se encontró modelo_breast_cancer_final.h5 en el directorio actual.')
except Exception as e:
    print('IPython no disponible o error al mostrar enlace:', e)

---
Fin del notebook `modelTraining.ipynb`.

Notas:
- Este notebook entrena la CNN 
 y guarda el modelo final en `modelo_breast_cancer_final.h5`.
- No borra ni modifica otras carpetas de datos.
